# Bayesian PCA

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris

In [2]:
%config InlineBackend.figure_format = "retina"
np.set_printoptions(precision=4, suppress=True)

* A fully probabilistic approach to PCA allows us to automatically choose the dimensionality of the principal subspace
* In this notebook, we consider a model in which only $\bf W$ has a prior distribution of the form

$$
    p({\bf W}\vert\boldsymbol\alpha) = \prod_{m=1}^M \left(\frac{\alpha_m}{2\pi}\right)^{D/2} \exp\left(-\frac{\alpha_m}{2}{\bf w}_m^T{\bf w}_m\right)
$$

To choose the values of $\{\alpha_m\}_{m=1}^M$, we maximize the marginal likelihood once $\bf W$ has been integrated out. That is, we want to maximize 

$$
    p({\bf X}\vert\boldsymbol\mu, \boldsymbol\mu, \sigma^2) = \int\prod_{m=1}^M \mathcal{N}({\bf w}_m\vert {\bf 0}, \alpha_m{\bf I}) \cdot \mathcal{N}({\bf x}_n \vert \boldsymbol\mu,{\bf C}) \ \text{d}{\bf W} \label{eq:a}\tag{1}
$$

Where
* ${\bf C} = {\bf W}{\bf W}^T + \sigma^2{\bf I}$


The integral over $\eqref{eq:a}$ is intractable, thus we make use of the Laplace approximation

In [3]:
iris = load_iris()
X_iris, y_iris = iris["data"], iris["target"]

In [4]:
N, D = X_iris.shape
M = 2
sigma2 = 1

In [5]:
W = np.random.randn(D, M)
S = np.cov(X_iris.T)
C = W @ W.T + np.eye(D) * sigma2
alpha = np.random.rand(M)

Rewriting the equation $\sum_{m}\alpha_m{\bf w}^T_m{\bf w}_m$

In [6]:
np.einsum("m,jm,jm",alpha, W, W)

7.1233544857254705

In [7]:
np.trace(alpha * np.identity(M) @ W.T @ W)

7.123354485725471

In [8]:
np.trace(W @ (alpha * np.identity(M)) @ W.T)

7.123354485725471

In [9]:
np.linalg.inv(S @ np.linalg.inv(C) - np.eye(D)) @ C @ W

array([[  9.24  , -18.6294],
       [ -0.6708,  24.698 ],
       [ -4.0795,  -1.6487],
       [  2.0436, -21.8892]])

In [10]:
A = np.random.randn(5, 5)
B = np.random.randn(5, 5)
C = np.random.randn(5, 5)

In [11]:
np.trace(A @ B)

3.151859574213212

In [12]:
np.einsum("in,ni->", A, B)

3.151859574213212

In [13]:
np.trace(A @ B @ A.T)

-6.112247548953237

In [14]:
np.einsum("nm,ml,nl->", A, B, A)

-6.112247548953242

$$
    \frac{\partial}{\partial A_{ij}}\text{Tr}(ABA^T) = \left(A\left[B^T + B\right]\right)_{ij}
$$

In [15]:
A @ (B.T  + B)

array([[ 3.3534, -2.5422,  1.4369,  4.3057,  1.789 ],
       [ 0.0697, -7.2008,  4.1166, -1.0726, -1.6574],
       [-0.2658,  0.7849, -3.2606, -2.8398, -1.8644],
       [ 6.6364, -0.1028, -4.5675, 16.3631,  4.7665],
       [-4.1316, -3.3403, -0.5746, -8.3594, -5.8196]])

In [16]:
np.einsum("il, jl", A, B) + np.einsum("il,lj", A, B)

array([[ 3.3534, -2.5422,  1.4369,  4.3057,  1.789 ],
       [ 0.0697, -7.2008,  4.1166, -1.0726, -1.6574],
       [-0.2658,  0.7849, -3.2606, -2.8398, -1.8644],
       [ 6.6364, -0.1028, -4.5675, 16.3631,  4.7665],
       [-4.1316, -3.3403, -0.5746, -8.3594, -5.8196]])

$$
    \text{Tr}\left(ABA^TC\right)
$$

In [17]:
np.trace(A @ B @ A.T @ C)

-11.805289028013819

In [18]:
np.einsum("nm,ml,ol,on->", A, B, A, C)

-11.805289028013789

$$
    \frac{\partial}{\partial A_{ij}}\text{Tr}\left(ABA^TC\right) = \left(C^T A B^T + C A B\right)_{ij}
$$

In [19]:
np.einsum("jn,mn,mi->ij", B, A, C)

array([[ 2.5339, -2.6197,  1.0428, 14.9636,  6.5248],
       [-0.7265, -4.1606,  0.5044, 13.231 ,  3.9848],
       [-0.6543, -2.3858, -1.6314,  4.5779,  0.9784],
       [-2.7903, -1.6052, -1.1076,  4.1098, -0.1175],
       [-7.2523, -5.1329,  3.7958, -8.5381, -8.1382]])

In [20]:
(C.T @ A @ B.T)

array([[ 2.5339, -2.6197,  1.0428, 14.9636,  6.5248],
       [-0.7265, -4.1606,  0.5044, 13.231 ,  3.9848],
       [-0.6543, -2.3858, -1.6314,  4.5779,  0.9784],
       [-2.7903, -1.6052, -1.1076,  4.1098, -0.1175],
       [-7.2523, -5.1329,  3.7958, -8.5381, -8.1382]])

In [21]:
np.einsum("nm,mj,in->ij", A, B, C)

array([[ 4.8074, -5.2833,  2.6995,  0.5159,  2.0417],
       [ 3.7879, -4.2632, -1.4116,  2.4091, -0.8741],
       [-7.598 ,  5.1263, -0.5171, -5.5249, -0.9244],
       [ 7.2606, -3.6446, -0.8113,  7.3453,  1.4012],
       [-2.4637,  0.7962, -1.827 , -2.6136, -2.1997]])

In [22]:
C @ A @ B

array([[ 4.8074, -5.2833,  2.6995,  0.5159,  2.0417],
       [ 3.7879, -4.2632, -1.4116,  2.4091, -0.8741],
       [-7.598 ,  5.1263, -0.5171, -5.5249, -0.9244],
       [ 7.2606, -3.6446, -0.8113,  7.3453,  1.4012],
       [-2.4637,  0.7962, -1.827 , -2.6136, -2.1997]])

In [24]:
import sympy
i, j, n, m, l, o = sympy.symbols("i j n m l o")
D = sympy.symbols("D")
A, B, C = sympy.symbols("A B C")

# References 

1. https://papers.nips.cc/paper/1549-bayesian-pca.pdf
2. https://www.cs.toronto.edu/~rsalakhu/STA4273_2015/notes/Lecture8_2015.pdf
3. https://haralick.org/ML/Neural_Networks_for_Pattern_Recognition_Christopher_Bishop.pdf
4. http://www.miketipping.com/papers/met-mppca.pdf
5. https://www.apps.stat.vt.edu/leman/VTCourses/PPCA.pdf